In [2]:
#!/usr/bin/env python
# -=-<[ Bismillahirrahmanirrahim ]>-=-
# -*- coding: utf-8 -*-
# @Date    : 2022-03-14 12:40:33
# @Author  : Dahir Muhammad Dahir (dahirmuhammad3@gmail.com)
# @Link    : link
# @Version : 1.0.0



import angr, claripy, monkeyhex

base_address = 0x400000

project = angr.Project("./otp", auto_load_libs=False, main_opts={'base_addr': base_address})
strncpy = project.loader.main_object.plt['strncpy']
project.hook(strncpy, angr.SIM_PROCEDURES['libc']['strncpy']())


In [3]:
key_arg_length = 100
key_args = []


for i in range(key_arg_length):
    key_args.append(claripy.BVS(f"x_{i}", 8))

key = claripy.Concat(*key_args)

state = project.factory.entry_state(args=["./otp", key])
# state = project.factory.entry_state()


In [4]:
for arg in key_args:
    # state.solver.add(arg >= 0x30)
    for x in range(0x1, 0x30):
        state.solver.add(arg != x)
    for i in range(0x3a, 0x61):
        state.solver.add(arg != i)
    state.solver.add(arg < 0x67)


In [5]:
sim_mgr = project.factory.simgr(state)
sim_mgr.explore(find=0x4009d9, avoid=[0x4009c6, 0x4007b9])

WARNING | 2022-03-15 15:16:47,144 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing memory or registers with an unspecified value. This could indicate unwanted behavior.
WARNING | 2022-03-15 15:16:47,144 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING | 2022-03-15 15:16:47,145 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING | 2022-03-15 15:16:47,145 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING | 2022-03-15 15:16:47,146 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING | 2022-03-15 15:16:47,146 | angr.storage.memory_mixins.default_filler_mixin | Filling memo

In [ ]:
# sim_mgr.deadended[0].posix.dumps(1)
# pruned_state = sim_mgr.pruned[0]

# print(hex(pruned_state.addr))

# for i in pruned_state.history.parents:
#     print(i)

NameError: name 'sim_mgr' is not defined